In [ ]:
import os
from pathlib import Path
import pickle
import sys
sys.path.append("../")

import contextily as cx
import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from sklearn import metrics
import statsmodels.api as sm


from openbustools import plotting, standardfeeds
from openbustools.traveltime import data_loader, model_utils

In [ ]:
kcm_res, kcm_out = model_utils.load_results("../results/kcm/")
atb_res, atb_out = model_utils.load_results("../results/atb/")
# mix_res, mix_out = model_utils.load_results("../results/mix/")

In [ ]:
plot_subset = ['FF', 'GRU', 'CONV', 'TRSF', 'DEEPTTE']
plot_metric = 'mape'
plot_bounds = [0, .5]
selected_model = "GRU"

In [ ]:
kcm_res['run'] = 'kcm'
atb_res['run'] = 'atb'
# mix_res['run'] = 'mix'
# all_res = pd.concat([kcm_res, atb_res, mix_res])
all_res = pd.concat([kcm_res, atb_res])
kcm_out['run'] = 'kcm'
atb_out['run'] = 'atb'
# mix_out['run'] = 'mix'
# all_out = pd.concat([kcm_out, atb_out, mix_out])
all_out = pd.concat([kcm_out, atb_out])

all_res.loc[all_res['model']=='FF_TUNED', 'model'] = 'FF'
all_res.loc[all_res['model']=='GRU_TUNED', 'model'] = 'GRU'
all_res.loc[all_res['model']=='CONV_TUNED', 'model'] = 'CONV'
all_res.loc[all_res['model']=='TRSF_TUNED', 'model'] = 'TRSF'
all_res.loc[all_res['model']=='DEEPTTE_TUNED', 'model'] = 'DEEPTTE'

all_res

### Model Selection Baseline Results

In [ ]:
fig, axes = plt.subplots(1,1)
subset = all_res[all_res['metric']==plot_metric].copy()
subset = subset[subset['is_tuned']==False]
subset = subset[subset['experiment_name'].isin(['same_city'])]
subset = subset[subset['model'].isin(plot_subset)]
sns.barplot(ax=axes, data=subset[subset['is_tuned']==False], x='value', y='model', hue='run', palette=plotting.PALETTE).set_title('Baseline Models')
fig.tight_layout()
axes.set_xlim(plot_bounds)

In [ ]:
subset.groupby(['run','model'])[['value']].agg(['mean','std']).sort_values(['run', ('value','mean')], ascending=False)

### Model Selection Generalization Results

In [ ]:
fig, axes = plt.subplots(1,1)
fig.set_figheight(5)
fig.set_figwidth(8)
subset = all_res[all_res['metric']==plot_metric].copy()
subset = subset[subset['is_tuned']==False]
subset = subset[subset['experiment_name'].isin(['diff_city'])]
subset = subset[subset['model'].isin(plot_subset)]
sns.barplot(ax=axes, data=subset[subset['is_tuned']==False], x='value', y='model', hue='run', palette=plotting.PALETTE)
fig.tight_layout()
axes.set_title('Baseline Models Generalization')
axes.set_xlim([0, 1.0])
axes.set_xlabel('MAPE')
axes.set_ylabel('')
axes.legend(title='Run', handles=axes.get_legend().legend_handles, labels=['KCM', 'AtB', 'Mixed'])

In [ ]:
subset.groupby(['run','model'])[['value']].agg(['mean','std']).sort_values(['run', ('value','mean')], ascending=False)

### Model Selection Hyperparameter Results

In [ ]:
# kcm_hyper = model_utils.load_hyper_results("../logs/kcm_hyper_search/", "GRU")
# kcm_hyper = kcm_hyper.drop_duplicates(['model', 'version'])

# kcm_hyper_res, atb_hyper_out = model_utils.load_results("../results/kcm_hyper_search/")
# kcm_hyper_res['version'] = kcm_hyper_res['model'].str.split("_").str[2:4].str.join("_")
# kcm_hyper_res = kcm_hyper_res.groupby(['version', 'experiment_name', 'metric'], as_index=False)[['value']].mean()

# hyper_res = pd.merge(kcm_hyper_res, kcm_hyper, on='version')
# hyper_res = hyper_res[hyper_res['metric']=='mape']
# hyper_res[hyper_res['experiment_name']=='diff_city']

In [ ]:
# atb_hyper = model_utils.load_hyper_results("../logs/atb_hyper_search/", "GRU")
# atb_hyper = atb_hyper.drop_duplicates(['model', 'version'])

# atb_hyper_res, atb_hyper_out = model_utils.load_results("../results/atb_hyper_search/")
# atb_hyper_res['version'] = atb_hyper_res['model'].str.split("_").str[2:4].str.join("_")
# atb_hyper_res = atb_hyper_res.groupby(['version', 'experiment_name', 'metric'], as_index=False)[['value']].mean()

# hyper_res = pd.merge(atb_hyper_res, atb_hyper, on='version')
# hyper_res = hyper_res[hyper_res['metric']=='mape']
# hyper_res[hyper_res['experiment_name']=='diff_city']

### Internal Generalization Results

In [ ]:
fig, axes = plt.subplots(2,1)
axes = axes.flatten()
fig.set_figheight(5)
fig.set_figwidth(8)
subset = all_res[all_res['metric']==plot_metric].copy()
subset = subset[subset['is_tuned']==False]
subset = subset[subset['experiment_name'].isin(['holdout', 'same_city'])]
subset = subset[subset['model'].isin([selected_model, 'AVG', 'SCH', 'DEEPTTE'])]
sns.barplot(ax=axes[0], data=subset[subset['run']=='kcm'], x='value', y='model', hue='experiment_name', palette=plotting.PALETTE)
sns.barplot(ax=axes[1], data=subset[subset['run']=='atb'], x='value', y='model', hue='experiment_name', palette=plotting.PALETTE)
# fig.tight_layout()
axes[0].set_xlim([0, 1.0])
axes[0].set_title('Internal Generalization')
axes[0].set_xlabel('')
axes[0].set_ylabel('Source City: KCM')
axes[0].get_legend().remove()
axes[1].set_xlim([0, 1.0])
axes[1].set_xlabel('MAPE')
axes[1].set_ylabel('Source City: AtB')
axes[1].legend(handles=axes[0].get_legend_handles_labels()[0], loc='lower right', labels=['Source City', 'Source City Holdout Routes'], ncol=1)
fig.savefig("../plots/internal_generalization.png", dpi=300)

In [ ]:
subset.groupby(['run','model','experiment_name'])[['value']].agg(['mean','std'])

In [ ]:
source_df = pd.read_pickle("../data/kcm_realtime/processed/analysis/2023_03_13.pkl")
source_df = source_df[~source_df['route_id'].isin(data_loader.HOLDOUT_ROUTES)]
# sns.histplot(source_df['calc_speed_m_s'])
np.mean(source_df['calc_speed_m_s']), np.std(source_df['calc_speed_m_s'])

In [ ]:
source_df = pd.read_pickle("../data/atb_realtime/processed/analysis/2023_03_13.pkl")
source_df = source_df[~source_df['route_id'].isin(data_loader.HOLDOUT_ROUTES)]
# sns.histplot(source_df['calc_speed_m_s'])
np.mean(source_df['calc_speed_m_s']), np.std(source_df['calc_speed_m_s'])

In [ ]:
# gtfs = standardfeeds.get_gtfs_shapes_lookup("../data/kcm_static/2023_01_23/")
# gtfs_shapes = standardfeeds.get_gtfs_shapes("../data/kcm_static/2023_01_23/", epsg=32148)

# fig, axes = plt.subplots(1,2)
# fig.set_figheight(5)
# fig.set_figwidth(10)
# axes = axes.flatten()

# gtfs_shapes.drop_duplicates('route_id').plot(ax=axes[0])
# gtfs_shapes[gtfs_shapes['route_id'].isin(data_loader.HOLDOUT_ROUTES)].plot(column='route_id', ax=axes[0])
# cx.add_basemap(ax=axes[0], crs=gtfs_shapes.crs.to_string(), alpha=0.6, source=cx.providers.MapBox(accessToken=os.getenv(key="MAPBOX_TOKEN")))

# subset = all_res[all_res['metric']==plot_metric]
# subset = subset[subset['run']=='kcm']
# subset = subset[subset['is_tuned']==False]
# subset = subset[subset['model_archetype'].isin(plot_subset)]
# subset = subset[subset['experiment_name'].isin(['same_city','holdout'])]
# sns.barplot(subset, x='value', y='model', hue='experiment_name', palette=plotting.PALETTE, ax=axes[1])

# fig.suptitle("Holdout Routes KCM")
# fig.tight_layout()
# axes[1].set_xlim(plot_bounds)

### External Generalization Results

In [ ]:
subset = all_res[all_res['metric']==plot_metric].copy()
subset = subset[subset['is_tuned']==False]
subset = subset[subset['experiment_name'].isin(['diff_city', 'same_city'])]
subset = subset[subset['model'].isin([selected_model, 'AVG', 'DEEPTTE'])]

# AtB models
subset_atb = subset[subset['run']=='atb'].copy()
subset_atb = subset_atb[subset_atb['experiment_name']=='same_city']
subset_kcm = subset[subset['run']=='kcm'].copy()
subset_kcm = subset_kcm[subset_kcm['experiment_name']=='diff_city']
subset_atb_gen = pd.concat([subset_atb, subset_kcm])

# KCM models
subset_kcm = subset[subset['run']=='kcm'].copy()
subset_kcm = subset_kcm[subset_kcm['experiment_name']=='same_city']
subset_atb = subset[subset['run']=='atb'].copy()
subset_atb = subset_atb[subset_atb['experiment_name']=='diff_city']
subset_kcm_gen = pd.concat([subset_kcm, subset_atb])

In [ ]:
fig, axes = plt.subplots(2,1)
fig.set_figheight(5)
fig.set_figwidth(8)
axes = axes.flatten()
sns.barplot(ax=axes[0], data=subset_kcm_gen, x='value', y='model', hue='experiment_name', palette=plotting.PALETTE)
sns.barplot(ax=axes[1], data=subset_atb_gen, x='value', y='model', hue='experiment_name', palette=plotting.PALETTE)
# fig.tight_layout()
axes[0].set_xlim(plot_bounds)
axes[0].set_title('External Generalization')
axes[0].set_xlabel('')
axes[0].set_ylabel('Source City: KCM')
axes[0].get_legend().remove()
axes[1].set_xlim(plot_bounds)
axes[1].set_xlabel('MAPE')
axes[1].set_ylabel('Source City: AtB')
axes[1].legend(handles=axes[0].get_legend_handles_labels()[0], loc='lower right', labels=['Tested on Source City', 'Tested on Target City'], ncol=1, title='')
fig.savefig("../plots/external_generalization.png", dpi=300)

### Tuning Results

In [ ]:
# Show dist of improvement rather than bars
subset = all_res[all_res['metric']==plot_metric].copy()
subset = subset[subset['experiment_name'].isin(['diff_city'])]
subset = subset[subset['model'].isin([selected_model, 'DEEPTTE'])]

subset_notune = subset[subset['is_tuned']==False]
subset_tune = subset[subset['is_tuned']==True]

subset = pd.merge(subset_notune, subset_tune, on=['run','model','fold'], suffixes=('_notune', '_tune'))
subset['pct_improvement'] = (subset['value_tune'] - subset['value_notune']) / subset['value_notune']
subset['net_improvement'] = (subset['value_tune'] - subset['value_notune'])

In [ ]:
fig, axes = plt.subplots(2,1)
fig.set_figheight(5)
fig.set_figwidth(8)
axes = axes.flatten()
sns.boxplot(ax=axes[0], data=subset[subset['run']=='kcm'], x='net_improvement', hue='model', palette=plotting.PALETTE)
sns.boxplot(ax=axes[1], data=subset[subset['run']=='atb'], x='net_improvement', hue='model', palette=plotting.PALETTE)
# fig.tight_layout()
axes[0].set_xlim([-.3, .1])
axes[0].set_title('Tuned Improvements to External Generalization')
axes[0].set_xlabel('')
axes[0].set_ylabel('Source City: KCM')
axes[0].get_legend().remove()
axes[0].set_xticklabels(['-30%', '-25%', '-20%', '-15%', '-10%', '-5%', '0%', '5%', '10%'])
axes[1].set_xlim([-.3, .1])
axes[1].set_xlabel('Net Reduction in MAPE w/100 Tuning Samples')
axes[1].set_ylabel('Source City: AtB')
axes[1].legend(handles=axes[0].get_legend_handles_labels()[0], loc='upper right', labels=['GRU', 'DeepTTE'], ncol=1)
axes[1].set_xticklabels(['-30%', '-25%', '-20%', '-15%', '-10%', '-5%', '0%', '5%', '10%'])
fig.savefig("../plots/tuned_generalization.png", dpi=300)

### Ablation Results

In [ ]:
subset = all_res[all_res['metric']==plot_metric].copy()
subset = subset[subset['experiment_name']=='same_city']
subset = subset[subset['is_tuned']==True]
subset = subset[subset['model_archetype'].isin(['GRU'])]
subset.groupby(['run','model'])[['value']].agg(['mean','std'])#.sort_values(['run', ('value','mean')], ascending=False)

In [ ]:
subset = all_res[all_res['metric']==plot_metric].copy()
subset = subset[subset['experiment_name']=='diff_city']
# subset = subset[subset['is_tuned']==False]
subset = subset[subset['model_archetype'].isin(['GRU'])]
subset.groupby(['run','model','is_tuned'])[['value']].agg(['mean','std'])#.sort_values(['run', ('value','mean')], ascending=False)

### Residuals

In [ ]:
# fig, axes = plt.subplots(3,3)
# fig.set_figheight(8)
# fig.set_figwidth(12)

# subset = all_out[all_out['model']==selected_model]
# subset = subset[subset['experiment_name']=='same_city']
# subset = subset.sample(10000)

# for i, run_name in enumerate(['kcm','atb','mix']):
#     plot_df = subset[subset['run']==run_name]
#     sns.residplot(plot_df, ax=axes[i,0], x='labels', y='preds', lowess=True, scatter_kws={'marker': '.'}, line_kws={'color': 'red'})
#     sm.qqplot(plot_df['residuals'], ax=axes[i,1], dist=stats.t, distargs=(len(plot_df)-1,), line='45', fit=True)
#     sns.histplot(plot_df['residuals'], ax=axes[i,2], bins=100)
#     axes[i,0].set_xlim(0,3000)
#     axes[i,1].set_ylim(-10,10)
#     axes[i,2].set_xlim(-500,500)
#     axes[i,0].set_title(run_name)

# fig.suptitle(f"Same City Residuals - {selected_model}")
# fig.tight_layout()

In [ ]:
# fig, axes = plt.subplots(3,3)
# fig.set_figheight(8)
# fig.set_figwidth(12)

# subset = all_out[all_out['model']==selected_model]
# subset = subset[subset['experiment_name']=='diff_city']
# subset = subset.sample(10000)

# for i, run_name in enumerate(['kcm','atb','mix']):
#     plot_df = subset[subset['run']==run_name]
#     sns.residplot(plot_df, ax=axes[i,0], x='labels', y='preds', lowess=True, scatter_kws={'marker': '.'}, line_kws={'color': 'red'})
#     sm.qqplot(plot_df['residuals'], ax=axes[i,1], dist=stats.t, distargs=(len(plot_df)-1,), line='45', fit=True)
#     sns.histplot(plot_df['residuals'], ax=axes[i,2], bins=100)
#     axes[i,0].set_xlim(0,3000)
#     axes[i,1].set_ylim(-10,10)
#     axes[i,2].set_xlim(-500,500)
#     axes[i,0].set_title(run_name)

# fig.suptitle(f"Different City Residuals - {selected_model}")
# fig.tight_layout()

In [ ]:
# fig, axes = plt.subplots(3,3)
# fig.set_figheight(8)
# fig.set_figwidth(12)

# subset = all_out[all_out['model']==selected_model]
# subset = subset[subset['experiment_name']=='holdout']
# subset = subset.sample(1000)

# for i, run_name in enumerate(['kcm','atb','mix']):
#     plot_df = subset[subset['run']==run_name]
#     sns.residplot(plot_df, ax=axes[i,0], x='labels', y='preds', lowess=True, scatter_kws={'marker': '.'}, line_kws={'color': 'red'})
#     sm.qqplot(plot_df['residuals'], ax=axes[i,1], dist=stats.t, distargs=(len(plot_df)-1,), line='45', fit=True)
#     sns.histplot(plot_df['residuals'], ax=axes[i,2], bins=100)
#     axes[i,0].set_xlim(0,3000)
#     axes[i,1].set_ylim(-10,10)
#     axes[i,2].set_xlim(-500,500)
#     axes[i,0].set_title(run_name)

# fig.suptitle(f"Holdout Residuals - {selected_model}")
# fig.tight_layout()